# Class를 관리하는법
1. 딕셔너리 튜플보다는 헬퍼 클래스를 관리하자
2. 인터페이스가 간단하면 클래스 대신 함수를 활용하자(후크함수)
3. 객체를 범용으로 이용하려면 @classmethod 다형성을 이용하자
4. super로 부모 클래스를 초기화하자
5. 공개 속성과 비공개 속성
6. 커스텀 컨테이너 타입은 collections.abc를 활용하자

## 1. 딕셔너리 튜플보다는 헬퍼 클래스를 관리하자



In [4]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades)/ len(grades)

In [5]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
print(book.average_grade('Isaac Newton'))

90.0


In [6]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grades_list= by_subject.setdefault(subject, [])
        grades_list.append(grade)
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count= 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [7]:
book = BySubjectGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton','Math', 90)
book.report_grade('Isaac Newton','Math', 80)
book.report_grade('Isaac Newton','Gym', 70)
book.report_grade('Isaac Newton','Gym', 60)
print(book.average_grade('Isaac Newton'))

75.0


```
class WeightedGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grades_list= by_subject.setdefault(subject, [])
        grades_list.append((score, weight))
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count= 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                
                score_sum += sum(score)
                score_count += len(weight)
        return total / count
```

이정도가 되면 딕셔너리와 튜플 대신 클래스의 계층구조를 사용할 때가 된 것이다.

### 클래스 리팩토링

관리하기 복잡하다고 느끼는 즉시 클래스로 옮겨야한다. 그러면 데이터를 더 잘 캡슐화한 잘 정의된 인터페이스를 제공할 수 있다.  
의존 관계에서 __가장 아래에 있는 성적__ 부터 클래스를 옮겨보자. 이렇게 간단한 정보를 담기에 클래스는 너무 무거워 보인다.  
성적은 변하지 않으니 튜플을 사용하는 게 더 적절해보인다. 다음 코드에서는 리스트안에 성적을 기록하려고 __(score, weight)__ 튜플을 사용한다.

In [11]:
grades=[]
grades.append((95, 0.45))

total = sum(score*weight for score, weight in grades)
total_weight = sum(weight for _,weight in grades)
average_grade = total / total_weight
print(average_grade)

95.0


튜플에 이름까지 추가하면, 다음과 같은 코드로 나타낼 수 있다.  
이 때 튜플을 점점 더 길게 확장하는 패턴은 딕셔너리의 계층을 깊게 두는 방식과 비슷하다.

In [12]:
grades=[]
grades.append((95, 0.45, 'Great job'))
total = sum(score*weight for score, weight, _ in grades)
total_weight = sum(weight for _, weight, _ in grades)
average_grade = total / total_weight
print(average_grade)

95.0


이 때 이 방식은 `collections.namedtuple` 를 활용하면 더욱 손쉽게 다룰 수 있다.  
namedtuple이 여러상황에서 유용하긴 하지만 장점보다 단점을 만들어낼 수 있는 상황도 이해해야한다.  
namedtuple의 제약은 다음과 같다.  
- namedtuple로 만든 클래스에 기본 인수 값을 설정할 수 없다. 그래서 데이터에 선택적인 속성이 많으면 다루기 힘들어진다.
- 여전히 인덱스와 순회방법으로 접근 가능하나, 나중에 실제 클래스로 바꾸기가 어려운 경우가 다수 발생한다.

In [9]:
import collections
Grade = collections.namedtuple('Grade',('score', 'weight'))

이러한 상황들을 조합하여, 성적입력에 적합한 클래스를 만들어보면 다음과 같이 세 개의 클래스를 통해 진행할 수 있다.  
특히 마지막의 Gradebook 클래스의 경우 두 가지 클래스를 합쳐서 보관하는 역할을 한다.

In [23]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grad(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
        return total / total_weight

In [24]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in  self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    

In [25]:
class Gradebook(object):
    def __init__(self):
        self._students = {}
        
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [26]:
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grades(80, 0.10)

print(albert.average_grade())

AttributeError: 'Subject' object has no attribute 'report_grades'